# "Scrip para Descargar Datos del SDO con Consideración de la Rotación Diferencial del Sol desde una Base de Datos en Formato JSON"

In [1]:
import pandas as pd 
from astropy.time import Time
from sunpy.net import Fido
from sunpy.net import attrs as a
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
import sunpy.map
import glob
import os

import warnings

warnings.filterwarnings("ignore")

In [ ]:
def get_query(item, bottom_left, top_right, start_time, ends_time):
    """
    Returns a query object based on the specified item and parameters.

    The function checks if the item is within predefined cadence lists (aia_cad_12, aia_cad_24)
    or if it matches 'hmi'. Depending on the case, it creates and returns a query object
    with appropriate parameters.

    Parameters:
    - item (str)            : The item to check in the cadence lists or for 'hmi' match.
    - bottom_left, top_right: Coordinates for the cutout area.
    - start_time, ends_time : Time range for the query.

    Returns:
    - query: A search query object or None if the item doesn't match any criteria.
    """

    # Define the cadence lists for AIA
    aia_cad_12 = [94, 131, 171, 193, 211, 304, 335]
    aia_cad_24 = [1600, 1700]

    # Check if the item is in the aia_cad_12 list
    if item in aia_cad_12:
        cadence = 12
        cutout = a.jsoc.Cutout(bottom_left, top_right=top_right, tracking=True)
        query = Fido.search(
            a.Time(start_time, ends_time),
            a.Wavelength(item*u.angstrom),
            a.Sample(cadence*u.second),
            a.jsoc.Series.aia_lev1_euv_12s,
            a.jsoc.Notify('fjordoneza@unal.edu.co'),
            a.jsoc.Segment.image,
            cutout)

    # Check if the item is in the aia_cad_24 list
    elif item in aia_cad_24:
        cadence = 24
        cutout = a.jsoc.Cutout(bottom_left, top_right=top_right, tracking=True)
        query = Fido.search(
            a.Time(start_time, ends_time),
            a.Wavelength(item*u.angstrom),
            a.Sample(cadence*u.second),
            a.jsoc.Series('aia.lev1_uv_24s'),
            a.jsoc.Notify('fjordoneza@unal.edu.co'),
            a.jsoc.Segment.image,
            cutout)

    # Check if the item matches 'hmi' (case insensitive)
    elif item.lower() == 'hmi':
        cutout = a.jsoc.Cutout(bottom_left, top_right=top_right, tracking=True)
        cadence = 45
        query = Fido.search(
            a.Time(start_time, ends_time),
            a.Sample(cadence*u.second),
            a.jsoc.Series('hmi.M_45s'),
            a.jsoc.Notify('fjordoneza@unal.edu.co'),
            a.jsoc.Segment.magnetogram,
            cutout)

    else:
        # Handle the case where the item is not in any list
        query = None

    return query

## Reading parameters 

In [2]:
parameters_d06 = pd.read_excel('parameters_d06.xlsx')
parameters_d06 

,ALMA,Unnamed: 1,SDO,Unnamed: 3
0,Tx_helioprojective,-128.0,t_0_SDO\n(YYYY-MM-DDTHH:MM:SS),2018-04-12 15:52:01
1,Ty_helioprojective,-400.0,t_f_SDO\n(YYYY-MM-DDTHH:MM:SS),2018-04-12 16:25:20
2,x_size_snapshot,253.0,radio_crop_sdo,68.31
3,y_size_snapshot,253.0,NaN,NaN
4,ALMA_pix_size (arcsec),0.3,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,"Must t_0, t_f must be chosen \nIn the same ALM...",NaN,NaN,NaN
8,Radio_crop: were chosen so that it had\nA larg...,NaN,NaN,NaN


In [3]:
t_0, t_f = parameters_d06.iloc[0][3], parameters_d06.iloc[1][3]
Tx, Ty   = parameters_d06.iloc[0][1], parameters_d06.iloc[1][1]
aiasq    = parameters_d06.iloc[2][3]

In [4]:
start_time  = Time(t_0.isoformat(), scale='utc', format='isot')
ends_time   = Time(t_f.isoformat(), scale='utc', format='isot')

top_right   = SkyCoord((Tx+aiasq)*u.arcsec, (Ty+aiasq)*u.arcsec, obstime=start_time, observer="earth", frame="helioprojective")
bottom_left = SkyCoord((Tx-aiasq)*u.arcsec, (Ty-aiasq)*u.arcsec, obstime=start_time, observer="earth", frame="helioprojective")

In [6]:
# List of wavelengths to create directories for
wl = [94, 131, 171, 193, 211, 304, 335, 1600, 1700]
# Create the main 'SDO' directory if it does not exist
# The 'exist_ok=True' parameter allows the function to not raise an error if the directory already exists
os.makedirs('SDO', exist_ok=True)
# Loop through each wavelength in the list
for wavelength in wl:
    # Create a directory for each wavelength and its subdirectories ('crop' and 'crop_and_rotate')
    # The directories are created inside the 'SDO' directory
    os.makedirs(f'SDO/{wavelength}/crop', exist_ok=True)
    os.makedirs(f'SDO/{wavelength}/crop_and_rotate', exist_ok=True)
# Additionally, create directories for 'magt' and its subdirectories ('crop' and 'crop_and_rotate')
# These are also created inside the 'SDO' directory
os.makedirs('SDO/hmi/crop', exist_ok=True)
os.makedirs('SDO/hmi/crop_and_rotate', exist_ok=True)

In [7]:
data_for_download= [94, 131, 171, 193, 211, 304, 335, 1600, 1700, 'hmi']
#data_for_download= [1700, 'hmi']

In [17]:
for data_download in   data_for_download:
    item = data_download
    query_data= get_query(item, bottom_left, top_right, start_time, ends_time)
    query_data

    file_download = Fido.fetch(query_data, path='SDO/{}'.format(item))
    file_download =sorted(file_download)


    tmp = sunpy.map.Map(file_download)

    aia_seq_totated = [tmp[i].rotate(use_scipy=True, missing=0)  for i in range(len(tmp))]

    new_files_rotated = [ 'SDO/{}/crop_and_rotate/'.format(item.split('/')[1]) + item.split('/')[2][:-5] + '_crop_and_rotate.fits' 
                          for item in file_download]
    new_files         = ['SDO/{}/crop/'.format(item.split('/')[1]) + item.split('/')[2][:-5] + '_crop.fits'
                         for item in file_download]

    for img, file_name in zip(aia_seq_totated, new_files_rotated):
                img.save     (file_name, overwrite=True         )

    for img, file_name in zip(tmp, new_files           ):
                img.save     (file_name, overwrite=True)

Export request pending. [id=JSOC_20231125_210, status=2]
Waiting for 0 seconds...
3 URLs found for download. Full request totalling 1MB


Files Downloaded:   0%|          | 0/3 [00:00<?, ?file/s]

aia.lev1_uv_24s.2018-04-12T155147Z.1700.image.fits:   0%|          | 0.00/83.5k [00:00<?, ?B/s]

aia.lev1_uv_24s.2018-04-12T155211Z.1700.image.fits:   0%|          | 0.00/83.5k [00:00<?, ?B/s]

aia.lev1_uv_24s.2018-04-12T155235Z.1700.image.fits:   0%|          | 0.00/83.5k [00:00<?, ?B/s]

Export request pending. [id=JSOC_20231125_192, status=2]
Waiting for 0 seconds...
2 URLs found for download. Full request totalling 1MB


Files Downloaded:   0%|          | 0/2 [00:00<?, ?file/s]

hmi.m_45s.20180412_155230_TAI.2.magnetogram.fits:   0%|          | 0.00/104k [00:00<?, ?B/s]

hmi.m_45s.20180412_155315_TAI.2.magnetogram.fits:   0%|          | 0.00/104k [00:00<?, ?B/s]

In [15]:
item = 'hmi'
#item = 1700

query_data= get_query(item, bottom_left, top_right, start_time, ends_time)
file_download = Fido.fetch(query_data, path='SDO/crop/{}'.format(item))
file_download =sorted(file_download)

tmp = sunpy.map.Map(file_download)

aia_seq_totated = [tmp[i].rotate(use_scipy=True, missing=0)  for i in range(len(tmp))]

new_files_rotated = [ 'SDO/{}/crop_and_rotate/'.format(item.split('/')[1]) + item.split('/')[2][:-5] + '_crop_and_rotate.fits' 
                        for item in file_download]
#new_files         = ['SDO/{}/crop/'.format(item.split('/')[1]) + item.split('/')[2][:-5] + '_crop.fits'
#                        for item in file_download]

for img, file_name in zip(aia_seq_totated, new_files_rotated):
            img.save     (file_name, overwrite=True         )

for img, file_name in zip(tmp, new_files           ):
            img.save     (file_name, overwrite=True)

Export request pending. [id=JSOC_20231125_192, status=2]
Waiting for 0 seconds...
2 URLs found for download. Full request totalling 1MB


Files Downloaded:   0%|          | 0/2 [00:00<?, ?file/s]